In [15]:
using DifferentialEquations
using ParameterizedFunctions
using Sundials
using LSODA
using DataFrames
using CSV

In [16]:
function temp_gen(hour, temp_ref)
    temp_ref + hour / (20 * 24 * 365) + 10 * sin((2 * pi / 24) * hour) + 10 * sin((2 * pi / (24 * 365)) * hour)
end

function I_S(hour)
    0.001 + 0.0005 * sin((2 * pi / (24 * 365)) * hour) #Exogenous SOC input
end

function I_D(hour)
    0.0001 + 0.00005 * sin((2 * pi / (24 * 365)) * hour) #Exogenous DOC input
end

function arrhenius_temp_dep(parameter, temp, Ea, temp_ref)
    decayed_parameter = parameter * exp(-Ea / 0.008314 * (1 / temp - 1 / temp_ref))
end

function linear_temp_dep(parameter, temp, Q, temp_ref)
    modified_parameter = parameter - Q * (temp - temp_ref)
end

function analytical_steady_state_AWB_original(SOC_input, DOC_input, u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L, temp_ref)
    u_Q = linear_temp_dep(u_Q_ref, temp_ref, Q, temp_ref)
    V_D = arrhenius_temp_dep(V_D_ref, temp_ref, Ea_V_D, temp_ref)
    V_U = arrhenius_temp_dep(V_U_ref, temp_ref, Ea_V_U, temp_ref)    
    D₀ = -((K_U * (r_E + r_M)) / (r_E + r_M - u_Q * V_U))
    S₀ = -((K_D * r_L * (SOC_input * r_E * (u_Q - 1) - a_MSA * DOC_input * r_M * u_Q + SOC_input * r_M * (-1 + u_Q - a_MSA * u_Q))) / (DOC_input * u_Q * (-a_MSA * r_L * r_M + r_E * V_D) + SOC_input * (r_E * r_L * (u_Q - 1) + r_L * r_M * (-1 + u_Q - a_MSA * u_Q) + r_E * u_Q * V_D)))
    M₀ = -((u_Q * (SOC_input + DOC_input)) / ((r_E + r_M) * (u_Q - 1)))
    E₀ = -((r_E * u_Q * (SOC_input + DOC_input)) / (r_L * (r_E + r_M) * (u_Q - 1))) 
    return [S₀, D₀, M₀, E₀]
end

analytical_steady_state_AWB_original (generic function with 1 method)

In [17]:
function analytical_steady_state_AWB_full_ECA(SOC_input, DOC_input, u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L, temp_ref)
    #TO BE CONTINUED
end

analytical_steady_state_AWB_full_ECA (generic function with 1 method)

In [18]:
########################
###Non-param Constants##
########################

temp_ref = 283

#########################
##ODE System Parameters##
#########################

u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Separate parameters for full ECA
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50

50

In [29]:
###############
##ODE Solving##
###############

tspan = (0., 100000.) #in hours
C₀ = analytical_steady_state_AWB_original(I_S(0), I_D(0), u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L, temp_ref)
p = [u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L]

function AWB!(du, u, p, t)
    S, D, M, E = u
    u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L = p
    du[1] = dS = I_S(t) + a_MSA * r_M * M - ((arrhenius_temp_dep(V_D_ref, temp_gen(t, temp_ref), Ea_V_D, temp_ref) * E * S) / (K_D + S))
    du[2] = dD = I_D(t) + (1 - a_MSA) * r_M * M + ((arrhenius_temp_dep(V_D_ref, temp_gen(t, temp_ref), Ea_V_D, temp_ref) * E * S) / (K_D + S)) + r_L * E - ((arrhenius_temp_dep(V_U_ref, temp_gen(t, temp_ref), Ea_V_U, temp_ref) * M * D) / (K_U + D))
    du[3] = dM = linear_temp_dep(u_Q_ref, temp_gen(t, temp_ref), Q, temp_ref) * ((arrhenius_temp_dep(V_U_ref, temp_gen(t, temp_ref), Ea_V_U, temp_ref) * M * D) / (K_U + D)) - r_M * M - r_E * M
    du[4] = dE = r_E * M - r_L * E
end

AWB_prob = ODEProblem(AWB!, C₀, tspan, p)
AWB_sol = solve(AWB_prob, dt = 0.05, saveat = 0:10:100000) #sample result every 10 hours, forcing dt = 0.05 hours

retcode: Success
Interpolation: 1st order linear
t: 10001-element Array{Float64,1}:
      0.0
     10.0
     20.0
     30.0
     40.0
     50.0
     60.0
     70.0
     80.0
     90.0
    100.0
    110.0
    120.0
      ⋮
  99890.0
  99900.0
  99910.0
  99920.0
  99930.0
  99940.0
  99950.0
  99960.0
  99970.0
  99980.0
  99990.0
 100000.0
u: 10001-element Array{Array{Float64,1},1}:
 [53.60230547550432, 0.11420612813370475, 0.6707317073170732, 0.013414634146341465]
 [53.58781366128196, 0.11845764851991578, 0.6724130533455964, 0.01341470508088586]
 [53.59185268469108, 0.1179567412285703, 0.6717780637451978, 0.013414860343561584]
 [53.58577779960439, 0.1197242907192087, 0.6724850243213502, 0.013414961847224422]
 [53.57961678557137, 0.1210848139087266, 0.6732874864190576, 0.0134152118588256]
 [53.58359407989628, 0.12041433501836304, 0.6727193776315242, 0.013415415567302367]
 [53.56714478307691, 0.12446928473993384, 0.6747784667820815, 0.013415725101227191]
 [53.573493644059276, 0.12317457

In [30]:
##########################
##Debugging and Checking##
##########################

#Test if function sticks to steady state with no temperature forcing

function AWB_ss!(du, u, p, t)
    S, D, M, E = u
    u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L = p
    du[1] = dS = I_S(t) + a_MSA * r_M * M - (V_D_ref * E * S) / (K_D + S)
    du[2] = dD = I_D(t) + (1 - a_MSA) * r_M * M + (V_D_ref * E * S) / (K_D + S) + r_L * E - (V_U_ref * M * D) / (K_U + D)
    du[3] = dM = u_Q_ref * (V_U_ref * M * D) / (K_U + D) - r_M * M - r_E * M
    du[4] = dE = r_E * M - r_L * E
end

AWB_ss_prob = ODEProblem(AWB_ss!, C₀, tspan, p
AWB_ss_sol = solve(AWB_ss_prob)

function AWB_ss2!(du, u, p, t)
    S, D, M, E = u
    u_Q_ref, Q, a_MSA, K_D, K_U, V_D_ref, V_U_ref, Ea_V_D, Ea_V_U, r_M, r_E, r_L = p
    du[1] = dS = I_S(t) + a_MSA * r_M * M - ((arrhenius_temp_dep(V_D_ref, temp_ref, Ea_V_D, temp_ref) * E * S) / (K_D + S))
    du[2] = dD = I_D(t) + (1 - a_MSA) * r_M * M + ((arrhenius_temp_dep(V_D_ref, temp_ref, Ea_V_D, temp_ref) * E * S) / (K_D + S)) + r_L * E - ((arrhenius_temp_dep(V_U_ref, temp_ref, Ea_V_U, temp_ref) * M * D) / (K_U + D))
    du[3] = dM = linear_temp_dep(u_Q_ref, temp_ref, Q, temp_ref) * ((arrhenius_temp_dep(V_U_ref, temp_ref, Ea_V_U, temp_ref) * M * D) / (K_U + D)) - r_M * M - r_E * M
    du[4] = dE = r_E * M - r_L * E
end

AWB_ss2_prob = ODEProblem(AWB_ss2!, C₀, tspan, p)
AWB_ss2_sol = solve(AWB_ss2_prob)

retcode: Success
Interpolation: automatic order switching interpolation
t: 831-element Array{Float64,1}:
      0.0
      9.999999999999999e-5
      0.0010999999999999998
      0.011099999999999997
      0.11109999999999996
      1.1110999999999995
     11.111099999999993
     69.83787793494952
    168.68706850472046
    292.17664330371645
    463.91338645513343
    692.6381932457958
   1037.390427540858
      ⋮
  98460.25683273027
  98597.73313592507
  98772.38416540857
  98940.25438292598
  99098.61463755954
  99251.39772218523
  99398.33883024077
  99539.53513590647
  99675.13684326617
  99805.40137703366
  99930.65196519095
 100000.0
u: 831-element Array{Array{Float64,1},1}:
 [53.60230547550432, 0.11420612813370475, 0.6707317073170732, 0.013414634146341465]
 [53.60230547550432, 0.11420612813370493, 0.6707317073170732, 0.013414634146341465]
 [53.60230547550454, 0.11420612813372645, 0.6707317073170732, 0.013414634146341465]
 [53.602305475526414, 0.114206128135914, 0.6707317073170732, 

In [31]:
##################
##Exporting Data##
##################

df_AWB_soln = DataFrame(AWB_sol.u)
CSV.write("AWB_synthetic_soln.csv", df_AWB_soln)

"AWB_synthetic_soln.csv"